In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="meta-llama/llama-4-scout-17b-16e-instruct"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The best answer is  Neil Armstrong


In [8]:
%pip install -qU langchain_community beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/business-systems-analyst/job/R-60176")
page_data = loader.load().pop().page_content
print(page_data)






















Business Systems Analyst










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



S

In [19]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Business Systems Analyst",
  "experience": {
    "degree": "Bachelor's degree in Business Administration, Data Science, Analytics",
    "years": 5,
    "related occupation": "analyst-related occupation"
  },
  "skills": [
    "Data analysis",
    "Data modelling",
    "Data quality",
    "Business analysis",
    "Cognos",
    "Databricks",
    "SQL",
    "SparkSQL",
    "PL-SQL",
    "T-SQL",
    "Amazon S3",
    "Teradata",
    "Oracle DBMS"
  ],
  "description": "Perform technical analytics related to advanced data analysis, data mining, and source to target mapping; create and maintain complex context process and system diagrams; work with remote groups such as vendors, other Nike facilities, and or departments to gather and document tactical requirements from the business; coordinate and communicate functional requirements of business unit and technical solutions with IT capability resources by designing and documenting advanced functional specs; conform to sta

In [20]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Business Systems Analyst',
 'experience': {'degree': "Bachelor's degree in Business Administration, Data Science, Analytics",
  'years': 5,
  'related occupation': 'analyst-related occupation'},
 'skills': ['Data analysis',
  'Data modelling',
  'Data quality',
  'Business analysis',
  'Cognos',
  'Databricks',
  'SQL',
  'SparkSQL',
  'PL-SQL',
  'T-SQL',
  'Amazon S3',
  'Teradata',
  'Oracle DBMS'],
 'description': 'Perform technical analytics related to advanced data analysis, data mining, and source to target mapping; create and maintain complex context process and system diagrams; work with remote groups such as vendors, other Nike facilities, and or departments to gather and document tactical requirements from the business; coordinate and communicate functional requirements of business unit and technical solutions with IT capability resources by designing and documenting advanced functional specs; conform to standards, processes, and policies; apply global process stan

In [21]:
type(json_res)

dict

In [22]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [23]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [27]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https:

In [26]:
job = json_res
job['skills']

['Data analysis',
 'Data modelling',
 'Data quality',
 'Business analysis',
 'Cognos',
 'Databricks',
 'SQL',
 'SparkSQL',
 'PL-SQL',
 'T-SQL',
 'Amazon S3',
 'Teradata',
 'Oracle DBMS']

In [31]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Manav, a business development executive at A.B.C is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of A.B.C 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase A.B.C's portfolio: {link_list}
        Remember you are Manav, BDE at A.B.C. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking Efficiency with A.B.C's Tailored Solutions for Your Business Systems Analyst Needs

Dear Hiring Manager,

I came across the Business Systems Analyst position at your organization and was impressed by the scope of responsibilities, particularly in advanced data analysis, data mining, and source to target mapping. At A.B.C, we specialize in empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

Our expertise in AI and software consulting can help address your needs in data analysis, data modeling, and business analysis. With our proficiency in tools like Data analysis, Data modelling, Data quality, Business analysis, Cognos, Databricks, SQL, SparkSQL, PL-SQL, T-SQL, Amazon S3, Teradata, and Oracle DBMS, we can provide the technical expertise required for this role.

Some of our notable projects that demonstrate our capabilities include:

*   Machine learning and Python solutions: